## Supplementary figures: comparison SNPE with SMC-ABC (Allen Cell Type Database)

In [ ]:
%run -i common.ipynb

### load results

In [ ]:
list_cells_AllenDB = [[518290966,57,0.0234/126],[509881736,39,0.0153/184],[566517779,46,0.0195/198]]

n_post = len(list_cells_AllenDB)

# SNPE parameters
n_components = 1
n_sims = 25000
n_rounds = 2

# SMC-ABC parameters
n_particles = 1e3

posterior_ls = []
res_ls = []
weighted_samples_ls = []
for cell_num in range(n_post):

    ephys_cell = list_cells_AllenDB[cell_num][0]
    sweep_number = list_cells_AllenDB[cell_num][1]
    
    ##############################################################################
    # SNPE results
    filename1 = '../hodgkinhuxley/results/allen_'+str(ephys_cell)+'_'+str(sweep_number)+\
    '_run_1_round2_prior0013_param8_nosvi_ncomp'+str(n_components)+'_nsims'+str(n_sims*n_rounds)+'_snpe.pkl'
    filename2 = '../hodgkinhuxley/results/allen_'+str(ephys_cell)+'_'+str(sweep_number)+\
    '_run_1_round2_prior0013_param8_nosvi_ncomp'+str(n_components)+'_nsims'+str(n_sims*n_rounds)+'_snpe_res.pkl'
    
    _, _, posterior = io.load_pkl(filename1)
    res = io.load(filename2)
    
    posterior_ls.append(posterior[-1])
    res_ls.append(res)

    ##############################################################################
    # SMC-ABC results
    filename1 = '../hodgkinhuxley/results/allen_'+str(ephys_cell)+'_'+str(sweep_number)+\
    '_run_1_prior0013_param8_smc_abc.pkl'
    ps_smc, logweights_smc, eps_smc, all_nsims_smc = io.load_pkl(filename1)
        
    # weighted samples
    weights_smc = np.exp(logweights_smc)
    num_rep_samples = np.round(weights_smc[-1]*n_particles).astype('int')
    weighted_samples = np.repeat(ps_smc[-1],num_rep_samples, axis=0)
    weighted_samples_ls.append(weighted_samples)

### generate figures

In [ ]:
fig_inches = (SCALE_IN*mm2inches(FIG_WIDTH_MM), SCALE_IN*mm2inches(FIG_WIDTH_MM))

for cell_num in range(n_post):
    
    SUPP = 'supp_fig_hh_allen_smc_abc_cell'+str(cell_num)  # name for appendix figure
    SUPP_FIG_SVG = PATH_DROPBOX_FIGS + SUPP + '.svg'
    SUPP_FIG_PDF = PATH_DROPBOX_FIGS + SUPP + '.pdf'

    prior_min = res_ls[cell_num].generator.prior.lower
    prior_max = res_ls[cell_num].generator.prior.upper

    prior_lims = np.concatenate((prior_min.reshape(-1,1),
                                 prior_max.reshape(-1,1)),
                                axis=1)

    with mpl.rc_context(fname=MPL_RC):
        fig = plt.figure(figsize=fig_inches)
        
        viz.plot_pdf(posterior_ls[cell_num], lims=prior_lims, samples=weighted_samples_ls[cell_num].T,
                     figsize=fig_inches,fontscale=0.6,labels_params=LABELS_HH, ticks=True,
                     col1=COL['ESS'], col2=COL['SNPE'])
        
        plt.savefig(SUPP_FIG_SVG, facecolor='None', transparent=True)  # the figure is saved as svg
        plt.close()
        
    svg(SUPP_FIG_SVG)
    !$INKSCAPE --export-pdf $SUPP_FIG_PDF $SUPP_FIG_SVG